## Selective feature induction

Polysemy often leads to errors in text classifiers. Similarly, polarity shifts can flip the meaning of a sentence while still using the same terms.

Here, we investigate ways to 
- identify terms that are used in ambiguous contexts
- introduce new features that resolve that ambiguity


Our solution is to use "selective non-linearity" to refine such features. This works as follows:
- Identify terms that are moderately indicative of the positive class.
- Collect all instances that contain the term.
- Create a decision tree to classify those instances 100% correctly.
- Replace the term feature value for all instances as follows:
  - if it is 0, it stays 0.
  - else, replace the feature value with the output of the classifier.
  
In this way, we allow non-linear feature value to better discriminate pos/neg instances, but we limit this non-linearity to avoid over-fitting.

**Related Work**

- Polarity shifting:
  - <http://lexitron.nectec.or.th/public/COLING-2010_Beijing_China/PAPERS/pdf/PAPERS072.pdf>
- Subsequence kerlens:
  - <http://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=6890481>

Consider this (real) misclassification error:

>truth=1 pred=0 text=it never fails to engage us .  
>never fails=0.676812 us=0.486345 it never=0.103713 it=0.0842794 engage=0.0705244|||fails=-0.964354 fails to=-0.539867 never=-0.269861 to=-0.23602 to engage=-0.0702657

In [1]:
from collections import Counter
import copy
import csv
import math
import numpy as np
import pickle
import re
from sklearn.cross_validation import KFold
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import chi2
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.tree import DecisionTreeClassifier, export_graphviz, _tree

import matplotlib.pyplot as plt
% matplotlib inline

In [2]:
def print_errors(truths, preds, X, raws, idx2clf, feature_names):
    for i, (truth, pred, raw) in enumerate(zip(truths, preds, raws)):
        if truth != pred:
            clf = idx2clf[i]
            coef = clf.coef_[0]
            print 'truth=%d pred=%d prob=%.5f\ntext=%s' % (truth, pred, clf.predict_proba(X[i])[0][int(pred)], raw.strip())
            dot = np.array(X[i])[0] * coef
            print ' '.join(['%s=%g' % (feature_names[i], dot[i]) for i in np.argsort(dot)[::-1][:5] if dot[i] != 0]) + '|||' + \
                   ' '.join(['%s=%g' % (feature_names[i], dot[i]) for i in np.argsort(dot)[:5] if dot[i] != 0])
            print


In [101]:
from sklearn.neighbors import KNeighborsClassifier
from numpy.linalg import norm
import utils
# See http://alexminnaar.com/time-series-classification-and-clustering-with-python.html

def my_dist(x, y, start=1000):
    """ Allow lists to contain a mix of floats (for coefficients) and ints representing
    special keywords (but, and, not). These special ints begin at value start."""
    if x < start and y < start:  # default L1 or L2 norm
        return (x-y)**2  # abs(x-y)
    elif x >= start and y >= start:  # both are special keywords.
        return 10. if x != y else 0.
    else:  # one is a keyword and one is a coefficient.
        return 10. 
    
def get_coef(tok, voc, coef):
    # polars = {}
    polars = {'but': 1000, 'not': 1001}
    if tok in polars:
        return polars[tok]
    elif tok not in voc:
        return 0.
    else:
        return coef[voc[tok]]
    
def make_time_series(raws, coef, vec):
    tker = vec.build_tokenizer()
    voc = vec.vocabulary_
    serieses = []
    for raw in raws:
        toks = tker(raw)
        series = np.array([get_coef(t, voc, coef) for t in toks])
        serieses.append(series)
    return serieses

class DTWClassifier(object):
    
    def __init__(self, dist_f, window, neighbors, vec, n_folds, weighted, rand):
        if neighbors > 1:
            print 'neighbors > 1 not yet implemented'
            return
        self.n_folds = n_folds
        self.dist_f = dist_f
        self.vec = vec
        self.rand = rand
        self.neighbors = neighbors
        self.weighted = weighted
        self.dtw = utils.DTW(dist=dist_f, window=window)
        self.window = window
        self.dist_f = dist_f
        
    def fit(self, X, y, raws):
        indices = []
        classifiers = []
        for train, test in KFold(len(y), n_folds=self.n_folds, shuffle=True, random_state=self.rand):
            indices.extend(test)
            clf = LogisticRegression()
            clf.fit(X[train], y[train])
            classifiers.append(clf)
        # TODO: average classifiers.
        avg_coef = np.average([clf.coef_[0] for clf in classifiers], axis=0)
        # avg_coef = 1. / (1. + np.exp(avg_coef))  # logistic
        self.series = make_time_series(np.array(raws)[indices], avg_coef, self.vec)
        self.labels = y[indices]
        self.raws = np.array(raws)[indices]
        self.logreg = LogisticRegression()
        self.logreg.fit(X, y)
        self.coef = self.logreg.coef_[0]  # 1. / (1. + np.exp(self.logreg.coef_[0]))  # logistic
        print 'logreg training acc=%g' % accuracy_score(self.logreg.predict(X), y)
    
    def predict(self, X, raws, y):
        
        test_series = make_time_series(raws, self.coef, self.vec)
        predictions = []
        for i, (test_s, raw) in enumerate(zip(test_series, raws)):
            min_dist=float('inf')
            min_j = -1
            count = 0
            for j, train_s in enumerate(self.series):
                if utils.lb_keogh(train_s, test_s, self.dist_f, r=self.window) < min_dist:
                    dist = self.dtw.distance(train_s, test_s)
                    if dist < min_dist:
                        min_dist=dist
                        min_j = j
                    count += 1
            predictions.append(self.labels[min_j])
            print '\ntesting (label=%d)\n%s' % (y[i], raw.strip())
            print 'closest (label=%d)\n%s' % (self.labels[min_j], self.raws[min_j].strip())
            print 'did %d DTW computations' % count
        return predictions

In [102]:
def do_polarity_expt_knn():
    rand = np.random.RandomState(123456)    
    DATA = '/data/polarity/rt-polaritydata'
    neg = open(DATA + '/rt-polarity.neg').readlines()
    pos = open(DATA + '/rt-polarity.pos').readlines()
    y = np.array([0] * len(neg) + [1] * len(pos))
    raw = np.array(neg + pos)
    vectorizer = CountVectorizer(ngram_range=(1, 1), decode_error='replace', max_df=1.0, min_df=2, binary=True)
    X = vectorizer.fit_transform(neg + pos).todense()
    # Downsample for testing
    sample = rand.choice(np.arange(len(y)), 1000)
    X = X[sample]
    y = y[sample]    
    feature_names = vectorizer.get_feature_names()
    print 'read %d instances and %d features; label distribution=%s' % (len(y), len(feature_names), Counter(y))
    accuracies = []
    for fi, (train, test) in enumerate(KFold(len(y), n_folds=5, shuffle=True, random_state=rand)):
        test = test[:10]
        clf = DTWClassifier(dist_f=my_dist, window=8, neighbors=1, vec=vectorizer, n_folds=10,
                            weighted=True, rand=rand)
        clf.fit(X[train], y[train], raw[train])
        pred = clf.predict(X[test], raw[test], y[test])
        accuracies.append(accuracy_score(pred, y[test]))
        print 'fold %d acc=%g' % (fi, accuracies[-1])
    print 'avg acc=%g' % np.mean(accuracies)

reload(utils)
do_polarity_expt_knn()

read 1000 instances and 10012 features; label distribution=Counter({1: 502, 0: 498})
logreg training acc=0.9975

testing (label=0)
[garbus] discards the potential for pathological study , exhuming instead , the skewed melodrama of the circumstantial situation .
closest (label=0)
there's no point in extracting the bare bones of byatt's plot for purposes of bland hollywood romance .
did 676 DTW computations

testing (label=0)
a visually flashy but narratively opaque and emotionally vapid exercise in style and mystification .
closest (label=1)
i never thought i'd say this , but i'd much rather watch teens poking their genitals into fruit pies !
did 291 DTW computations

testing (label=1)
while the performances are often engaging , this loose collection of largely improvised numbers would probably have worked better as a one-hour tv documentary .
closest (label=0)
stirs potentially enticing ingredients into an uneasy blend of ghost and close encounters of the third kind .
did 693 DTW compu

In [84]:
def do_polarity_expt():
    rand = np.random.RandomState(123456)    
    DATA = '/data/polarity/rt-polaritydata'
    neg = open(DATA + '/rt-polarity.neg').readlines()
    pos = open(DATA + '/rt-polarity.pos').readlines()
    y = np.array([0] * len(neg) + [1] * len(pos))
    raws = np.array(neg + pos)
    vectorizer = CountVectorizer(ngram_range=(1, 1), token_pattern=r'(?u)\b\w+\b', decode_error='replace',
                                 max_df=1.0, min_df=2, binary=True)
    X = vectorizer.fit_transform(neg + pos).todense()
    feature_names = vectorizer.get_feature_names()
    print 'read %d instances and %d features; label distribution=%s' % (len(y), len(feature_names), Counter(y))
    print 'feature_names=', feature_names[:10]
    indices = []
    clfs = []
    for train, test in KFold(len(y), n_folds=10, shuffle=True, random_state=rand):
        indices.extend(test)
        clf = LogisticRegression()
        clf.fit(X[train], y[train])
        clfs.append(clf)
    avg_coef = np.average([clf.coef_[0] for clf in clfs], axis=0)
    series = make_time_series(raws[indices], avg_coef, vectorizer)
    return series, raws[indices], y[indices]

series, raw, labels = do_polarity_expt()

read 10662 instances and 10047 features; label distribution=Counter({0: 5331, 1: 5331})
feature_names= [u'000', u'007', u'1', u'10', u'100', u'101', u'102', u'10th', u'11', u'110']


In [100]:
# Print out most similar docs.
# for doci in [42, 12]:  #[8466, 637, 42, 12]:
reload(utils)
dtw = utils.DTW(dist=my_dist, window=8)

for doci in [3203, 3201]:  # 3203
    distances = [dtw.distance(series[doci], s) for s in series]
    lbks = [utils.lb_keogh(series[doci], s, dist_f=my_dist, r=8) for s in series]
    indices = np.argsort(distances)
    print ('closest documents to idx %d\n%s %s\nlabel=%d\n' %
           (doci, raw[doci], ' '.join('%.2f' % s for s in series[doci]), labels[doci]))
    for i in indices[1:21]:
        print('idx=%d dist=%.4f lbk=%.4f\n%s %s\nlabel=%d\n' % (i, distances[i], lbks[i],
                                                       raw[i],
                                                      ' '.join('%.2f' % s for s in series[i]),
                                                      labels[i]))


closest documents to idx 3203
the effort is sincere and the results are honest , but the film is so bleak that it's hardly watchable . 
 -0.01 -0.12 0.15 0.19 0.37 -0.01 -0.65 0.00 0.72 1000.00 -0.01 0.31 0.15 -0.52 0.70 0.12 0.15 0.08 -0.68 0.88
label=0

idx=7020 dist=1.0424 lbk=0.1635
the film is all a little lit crit 101 , but it's extremely well played and often very funny . 
 -0.01 0.31 0.15 -0.25 0.16 -0.52 0.12 0.00 0.53 1000.00 0.15 0.08 -0.52 0.20 -0.32 0.37 0.30 -0.17 1.04
label=1

idx=2429 dist=1.0597 lbk=0.4803
the cast is uniformly excellent . . . but the film itself is merely mildly charming . 
 -0.01 0.20 0.15 -0.45 0.60 1000.00 -0.01 0.31 -0.22 0.15 -0.28 -1.16 1.00
label=0

idx=1200 dist=1.1679 lbk=0.0000
it's a very sincere work , but it would be better as a diary or documentary . 
 0.15 0.08 0.16 -0.17 0.19 0.34 1000.00 0.15 -0.41 0.05 -0.21 -0.11 0.16 -0.38 -0.40 0.71
label=0

idx=6992 dist=1.2080 lbk=0.5146
if it tried to do anything more , it would fail and perhap

**Examples**

```
12 interesting , but not compelling . 
[-0.12858005448526144, -0.13590465658688544, -0.1440977709374012, 0.92921433403940867] 
label= 0 

BAD:   
10099 formuliac , but fun . 
[0, -0.1444148288054018, 1.0599427099457501] 
label= 1 

GOOD:  
465 clever but not especially compelling . 
[0.0079044745193047015, -0.1444148288054018, -0.20212797226837867, 0.72154019776540368, 0.81067640973917521] 
label= 0 
```
Order matters...

Making "but" and "not" special tokens resolves this problem.

```
closest documents to idx=42
the effort is sincere and the results are honest , but the film is so bleak that it's hardly watchable . 
 -0.03 0.05 0.23 0.29 0.43 -0.03 -0.76 -0.05 0.93 1000.00 -0.03 0.40 0.23 -0.52 0.84 0.22 0.14 -0.45 1.10
label=0

GOOD
idx=2922 dist=0.1059
the cast is uniformly excellent . . . but the film itself is merely mildly charming . 
 -0.03 0.05 0.23 -0.27 0.63 1000.00 -0.03 0.40 -0.49 0.23 -0.03 -1.05 0.82
label=0


BAD
idx=6140 dist=0.1036
the film is all a little lit crit 101 , but it's extremely well played and often very funny . 
 -0.06 0.26 0.07 -0.46 -0.38 -0.04 0.00 0.42 1000.00 0.21 -0.50 0.21 -0.46 0.35 0.36 -0.30 1.22
label=1
```

Need to make (.1 - (-.1)) a bigger distance than (.2-.1). We'll try logistic function:

In [ ]:
def do_newsgroup_expt(categories, ntrees, expt_f):
    rand = np.random.RandomState(123456)    
    newsgroups_train = fetch_20newsgroups(subset='train',
                                          remove=('headers', 'footers', 'quotes'),
                                          categories=categories)
    newsgroups_test = fetch_20newsgroups(subset='test',
                                         remove=('headers', 'footers', 'quotes'),
                                         categories=categories)

    vectorizer = CountVectorizer(binary=True, min_df=3)
    X_train = vectorizer.fit_transform(newsgroups_train.data).todense()
    y_train = newsgroups_train.target
    X_test = vectorizer.transform(newsgroups_test.data).todense()
    y_test = newsgroups_test.target
    print 'train size=', len(y_train), 'test size=', len(y_test)    
    expt_f(X_train, y_train, X_test, y_test, vectorizer, ntrees, rand)

In [ ]:
do_newsgroup_expt(['comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware'], ntrees=100, expt_f=do_expt_cv)

In [ ]:
do_newsgroup_expt(['comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware'], ntrees=100, expt_f=do_expt)

In [ ]:
do_newsgroup_expt(['comp.graphics', 'comp.windows.x'], ntrees=100, expt_f=do_expt_cv)

In [ ]:
do_newsgroup_expt(['comp.graphics', 'comp.windows.x'], ntrees=100, expt_f=do_expt)

In [ ]:
do_newsgroup_expt(['sci.crypt', 'sci.electronics'], ntrees=100, expt_f=do_expt_cv)

In [ ]:
do_newsgroup_expt(['sci.crypt', 'sci.electronics'], ntrees=100, expt_f=do_expt)

In [ ]:
# IMDB
# Sample 1k train and 1k test.
def top_feats(clf, x, feature_names):
    dot = np.array(x[0])[0] * clf.coef_[0]
    return ' '.join(['%s=%g' % (feature_names[i], dot[i]) for i in np.argsort(dot)[::-1][:5]]) + '|||' + \
           ' '.join(['%s=%g' % (feature_names[i], dot[i]) for i in np.argsort(dot)[:5]])

def do_imdb_expt(ntrees, expt_f):
    rand = np.random.RandomState(123456)    
    data = pickle.load(open('/data/aclImdb/data.pkl'))
    sample = rand.choice(range(len(data[1].train.data)), size=1000, replace=False)
    vectorizer = CountVectorizer(binary=True, min_df=4)
    X_train = vectorizer.fit_transform(data[1].train.data[sample]).todense()
    y_train = data[1].train.target[sample]
    X_test = vectorizer.transform(data[1].test.data[sample]).todense()
    y_test = data[1].test.target[sample]
    print 'train size=', len(y_train), 'test size=', len(y_test)    
    feature_names = np.array(vectorizer.get_feature_names())

    X_test_new = expt_f(X_train, y_train, X_test, y_test, vectorizer, ntrees, rand)    
    clf = LogisticRegression()
    clf.fit(X_train, y_train)
    preds_og = clf.predict(X_test)
    preds_new = clf.predict(X_test_new)
    # old right, new wrong:
    print 'OLD RIGHT, NEW WRONG:\n'
    for i, (pred_og, pred_new, truth) in enumerate(zip(preds_og, preds_new, y_test)):
        text = data[1].test.data[sample][i]
        feats = X_test[i]
        if pred_og != pred_new and pred_og == truth:
            print '\n\npred_og=', pred_og, 'pred_new=', pred_new, 'truth=', truth, \
            'top_feats=\n', top_feats(clf, feats, feature_names), \
            '\ntop_feats new=\n', top_feats(clf, X_test_new[i], feature_names), \
            '\ntop_feats diff=\n', top_feats(clf, np.abs(X_test_new[i] - feats), feature_names), \
            '\ninstance=\n', data[1].test.data[sample][i]

    print '\nOLD WRONG, NEW RIGHT:\n'
    diff_sums = np.array([0] * len(feature_names))
    for i, (pred_og, pred_new, truth) in enumerate(zip(preds_og, preds_new, y_test)):
        text = data[1].test.data[sample][i]
        feats = X_test[i]
        if pred_og != pred_new and pred_new == truth:
            print '\n\npred=', pred_og, 'pred_new=', pred_new, 'truth=', truth, \
            'top_feats=\n', top_feats(clf, feats, feature_names), \
            '\ntop_feats new=\n', top_feats(clf, X_test_new[i], feature_names), \
            '\ntop_feats diff=\n', top_feats(clf, np.abs(X_test_new[i] - feats), feature_names), \
            '\ninstance=\n', data[1].test.data[sample][i]    
            diff_sums += np.abs(X_test_new[i] - feats).tolist()[0]
    print 'most changed features:'
    print '\n'.join(['%s %d' % (feature_names[i], diff_sums[i]) for i in np.argsort(diff_sums)[::-1] if diff_sums[i] > 0])


In [ ]:
do_imdb_expt(ntrees=100, expt_f=do_expt_cv)

In [ ]:
do_imdb_expt(ntrees=40, expt_f=do_expt)

**Results**
- Without retraining classifier:
  - Using top 900 features by chi2, test accuracy goes from 0.809524 to 0.845560
  - Somewhat non-monotonic increase in accuracy as number of features increases (e.g., accuracy at 1000 is somewhat less; .839125)

In [ ]:
def twokenize(string, lowercase=True, keep_punctuation=False, collapse_urls=True, collapse_mentions=True, collapse_numbers=True):
    if not string:
        return []
    if lowercase:
        string = string.lower()
    tokens = []
    if collapse_urls:
        string = re.sub('http\S+', 'THIS_IS_A_URL', string)
    if collapse_mentions:
        string = re.sub('@\S+', 'THIS_IS_A_MENTION', string)
    if collapse_numbers:
        string = re.sub('[0-9]+', '99', string)
    if keep_punctuation:
        tokens = string.split()
    else:
        tokens = re.sub('\W+', ' ', string).split()
    return tokens

def do_ecig_expt(ntrees, expt_f):
    rand = np.random.RandomState(123456)  
    y = []
    tweets = []
    for row in csv.DictReader(open('/data/2/uic/elaine/AllSamplesPositiveandNeutral.csv')):
        y.append(int(row['sent']))
        tweets.append(row['text'])
    y = np.array(y)
    vectorizer = CountVectorizer(decode_error='ignore', ngram_range=(1, 1), max_df=1.0, min_df=2, tokenizer=twokenize, binary=True)
    X = vectorizer.fit_transform(tweets).todense()
    feature_names = vectorizer.get_feature_names()
    print 'read %d instances and %d features; label distribution=%s' % (len(y), len(feature_names), Counter(y))
    print 'feature_names=', feature_names[:10]
    og_preds = []
    og_acc = []
    new_preds = []
    truths = []
    new_acc = []
    for train, test in KFold(len(y), n_folds=5, shuffle=True, random_state=rand):
        truths.extend(y[test])
        clf = LogisticRegression()
        clf.fit(X[train], y[train])
        og_pred = clf.predict(X[test])
        og_preds.extend(og_pred)
        og_acc.append(accuracy_score(y[test], og_pred))

        X_test, acc = expt_f(X[train], y[train], X[test], y[test], vectorizer, ntrees, rand)
        new_acc.append(acc)
    print 'og acc=%g' % np.mean(og_acc)
    print 'new acc=%g' % np.mean(new_acc)

    #y_train = newsgroups_train.target
    #X_test = vectorizer.transform(newsgroups_test.data).todense()
    #y_test = newsgroups_test.target
    #print 'train size=', len(y_train), 'test size=', len(y_test)    
    #expt_f(X_train, y_train, X_test, y_test, vectorizer, ntrees, rand)

In [ ]:
do_ecig_expt(20, do_expt)

In [ ]:
do_ecig_expt(20, do_expt_cv)